
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)





# Training Legal Assertion


## Setup

In [ ]:
from johnsnowlabs import *

import pandas as pd
import json
import os

spark = start_spark()

# CoNLL Data Prep 

# Data Prep 

In [2]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Legal/data/assertion_fin.csv

In [2]:
training_df = pd.read_csv('./assertion_fin.csv')

In [3]:
training_data = spark.createDataFrame(training_df)
training_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
|task_id|            sentence|tkn_start|tkn_end|               chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
|      1|The Swedish East ...|        1|      4|Swedish East Indi...|   ORG|           PAST|
|      1|The Swedish East ...|        6|      8|Svenska Ostindisk...| ALIAS|           PAST|
|      1|The Swedish East ...|       10|     10|                SOIC| ALIAS|           PAST|
|      1|The Swedish East ...|       14|     14|          Gothenburg|   LOC|           PAST|
|      1|The Swedish East ...|       15|     15|              Sweden|   LOC|           PAST|
|      1|The Swedish East ...|       17|     17|                1731|  DATE|           PAST|
|      1|The Swedish East ...|       25|     25|               China|   LOC|           PAST|
|      1|The Swedish East ...|       28|     29|            Far East| 

In [4]:
training_data.printSchema()

root
 |-- task_id: long (nullable = true)
 |-- sentence: string (nullable = true)
 |-- tkn_start: long (nullable = true)
 |-- tkn_end: long (nullable = true)
 |-- chunk: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- assertion_label: string (nullable = true)



In [5]:
%time training_data.count()

CPU times: user 1.58 ms, sys: 628 µs, total: 2.21 ms
Wall time: 477 ms


8050

In [6]:
(train_data, test_data) = training_data.randomSplit([0.9, 0.1], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 8050
Test Dataset Count: 797


In [7]:
train_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
|task_id|            sentence|tkn_start|tkn_end|               chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
|      1|"Stockholms-varve...|        6|      6|           Stockholm|   LOC|           PAST|
|      1|"The funny busine...|        5|      8|Swedish East Indi...|   ORG|           PAST|
|      1|             (1998).|        0|      0|                1998|  DATE|           PAST|
|      1|2.5 tonnes) and t...|       34|     34|              Sweden|   LOC|           PAST|
|      1|37. Gothenburg: R...|        2|      7|Royal Society of ...|   ORG|           PAST|
|      1|= Decline and fal...|       11|     11|                1806|  DATE|           PAST|
|      1|= Early attempts ...|        9|     11|  Swedish East India|   ORG|           PAST|
|      1|= Early attempts ...|       19|     19|            merchant| 

# Using RoBerta Embeddings

In [8]:
roberta_embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ]roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
Download done! Loading the resource.
[ — ]

2022-12-12 23:57:29.870928: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


In [9]:
document = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")

chunk = nlp.Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("doc_chunk")\
    .setChunkCol("chunk")\
    .setStartCol("tkn_start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(False)

token = nlp.Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')


We save the test data in parquet format to use in `AssertionDLApproach()`. 

In [10]:
assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    roberta_embeddings])

assertion_test_data = assertion_pipeline.fit(test_data).transform(test_data)

assertion_train_data = assertion_pipeline.fit(training_data).transform(training_data)

In [11]:
assertion_test_data.columns

['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

In [12]:
assertion_train_data.columns

['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

In [13]:
assertion_test_data.write.mode('overwrite').parquet('test_data.parquet')

assertion_train_data.write.mode('overwrite').parquet('train_data.parquet')

## Graph setup

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline. 

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [14]:
graph_folder= "./tf_graphs"

In [15]:
assertion_graph_builder =  legal.TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("assertion_label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(1200)\
    .setHiddenUnitsNumber(25)

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [16]:
scope_window = [50, 50]

assertionStatus = legal.AssertionDLApproach()\
    .setLabelCol("assertion_label")\
    .setInputCols("document", "doc_chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setEpochs(2)\
    .setStartCol("tkn_start")\
    .setEndCol("tkn_end")\
    .setMaxSentLen(1200)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs/')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setTestDataset(path="test_data.parquet", read_as='SPARK', options={'format': 'parquet'})\
    .setScopeWindow(scope_window)
    #.setValidationSplit(0.2)\    
    #.setDropout(0.1)\    

In [17]:
assertion_pipeline = nlp.Pipeline(
    stages = [
    #document,
    #chunk,
    #token,
    #embeddings,
    assertion_graph_builder,
    assertionStatus])

In [18]:
training_data.printSchema()

root
 |-- task_id: long (nullable = true)
 |-- sentence: string (nullable = true)
 |-- tkn_start: long (nullable = true)
 |-- tkn_end: long (nullable = true)
 |-- chunk: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- assertion_label: string (nullable = true)



In [19]:
assertion_train_data = spark.read.parquet('train_data.parquet')

In [ ]:
%%time
assertion_model = assertion_pipeline.fit(assertion_train_data)

Checking the results saved in the log file

In [21]:
log_files = os.listdir("./training_logs")
log_files

['.ipynb_checkpoints', 'AssertionDLApproach_6ecfb96cc36a.log']

In [23]:
with open("./training_logs/"+log_files[1]) as log_file:
    print(log_file.read())

Name of the selected graph: ./tf_graphs/assertion_graph.pb
Training started, trainExamples: 8050


Epoch: 0 started, learning rate: 0.001, dataset size: 8050
Done, 422.551802948 total training loss: 53.39654, avg training loss: 0.84756416, batches: 63
Quality on test dataset: 
time to finish evaluation: 38.52s
Total test loss: 2.1283	Avg test loss: 0.3040
label	 tp	 fp	 fn	 prec	 rec	 f1
PRESENT	 182	 20	 35	 0.9009901	 0.83870965	 0.8687351
POSSIBLE	 164	 25	 13	 0.8677249	 0.92655367	 0.89617485
FUTURE	 100	 9	 23	 0.9174312	 0.8130081	 0.86206895
PAST	 261	 36	 19	 0.8787879	 0.93214285	 0.90467936
tp: 707 fp: 90 fn: 90 labels: 4
Macro-average	 prec: 0.89123356, rec: 0.87760353, f1: 0.88436604
Micro-average	 prec: 0.88707656, rec: 0.88707656, f1: 0.88707656


Epoch: 1 started, learning rate: 9.5E-4, dataset size: 8050
Done, 419.519225305 total training loss: 16.742996, avg training loss: 0.26576185, batches: 63
Quality on test dataset: 
time to finish evaluation: 38.10s
Total test l

In [24]:
assertion_test_data = spark.read.parquet('test_data.parquet')

In [25]:
preds = assertion_model.transform(assertion_test_data).select('assertion_label','assertion.result')

preds.show()

1g6ax7jxec82paqzk7ivz9fx7
codes retrieved: ArrayBuffer(1g6ax7jxec82paqzk7ivz9fx7), product code(from property): 1g6ax7jxec82paqzk7ivz9fx7
1g6ax7jxec82paqzk7ivz9fx7
codes retrieved: ArrayBuffer(1g6ax7jxec82paqzk7ivz9fx7), product code(from property): 1g6ax7jxec82paqzk7ivz9fx7


+---------------+----------+
|assertion_label|    result|
+---------------+----------+
|           PAST| [PRESENT]|
|           PAST|    [PAST]|
|           PAST|    [PAST]|
|           PAST|    [PAST]|
|           PAST|    [PAST]|
|        PRESENT| [PRESENT]|
|        PRESENT| [PRESENT]|
|           PAST|[POSSIBLE]|
|        PRESENT|    [PAST]|
|        PRESENT| [PRESENT]|
|        PRESENT| [PRESENT]|
|           PAST|    [PAST]|
|           PAST|    [PAST]|
|        PRESENT| [PRESENT]|
|           PAST|    [PAST]|
|           PAST|    [PAST]|
|           PAST|    [PAST]|
|        PRESENT| [PRESENT]|
|        PRESENT| [PRESENT]|
|        PRESENT| [PRESENT]|
+---------------+----------+
only showing top 20 rows



In [26]:
preds_df = preds.toPandas()

In [27]:
preds_df["result"] = preds_df["result"].apply(lambda x: x[0] if len(x) else pd.NA)

preds_df.dropna(inplace=True)

preds_df

,assertion_label,result
0,PAST,PRESENT
1,PAST,PAST
2,PAST,PAST
3,PAST,PAST
4,PAST,PAST
...,...,...
792,POSSIBLE,POSSIBLE
793,POSSIBLE,POSSIBLE
794,POSSIBLE,POSSIBLE
795,POSSIBLE,POSSIBLE


In [28]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['assertion_label'], preds_df['result']))

              precision    recall  f1-score   support

      FUTURE       0.97      0.92      0.94       123
        PAST       0.95      0.92      0.94       278
    POSSIBLE       0.93      0.99      0.96       177
     PRESENT       0.91      0.93      0.92       217

    accuracy                           0.94       795
   macro avg       0.94      0.94      0.94       795
weighted avg       0.94      0.94      0.94       795



### Saving the trained model

In [ ]:
assertion_model.stages

In [ ]:
# Save a Spark NLP model
assertion_model.stages[-1].write().overwrite().save('Assertion')

import shutil
shutil.make_archive('Assertion', 'zip', 'Assertion')